In [84]:
import os
import os.path as osp
import tensorflow as tf

In [85]:
def create_cnn():
    '''
    定义一个CNN分类模型,该模型的特征提取器(feature extractor)包含
    两层卷积,每层卷积后面通过Maxpooling降低分辨率;分类器(classifier)
    由一层全连接层fully-connected layers,FC)+一层输出层构成。
    '''
    #该CNN模型为串行模型
    model = tf.keras.models.Sequential()
    
    #=================== feature extractor ====================
    #第一层卷积需要指定输入图像的shape为[batchsize，32，32，3]
    #输出特征shape：[batchsize，32, 32, 32]
    model.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=(3,3),
                                     input_shape=(32, 32, 3),
                                     activation='relu',
                                     padding='same'))   
    
    
    #将特征图缩小一半，输出shape：[batchsize，16, 16, 32]
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    
    #第二层卷积输出特征shape：[batchsize，16, 16, 64]
    model.add(tf.keras.layers.Conv2D(filters=64,
                                     kernel_size=(3,3),
                                     input_shape=(32, 32, 3),
                                     activation='relu',
                                     padding='same'))
    
    
    #将特征图缩小一半，输出shape：[batchsize，8, 8, 64]
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    
    #将2D的特征图变为向量，输出shape：[batchsize, 4096]
    model.add(tf.keras.layers.Flatten())
    
    #===========--------- classifier ==========================
    #第一层FC， 输出shape:[batchsize, 1024]
    model.add(tf.keras.layers.Dense(1024, activation='relu'))
    
    #随机将第一层FC的30%weights值设置为0，缓解过拟合
    #输出shape：[batchsize，1024]
    model.add(tf.keras.layers.Dropout(rate=0.3))
    
    #第二层FC层作为输出层，10个类别的softmax概率
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    
    return model

In [86]:
def create_cnn():
    '''
    定义一个CNN分类模型,该模型的特征提取器(feature extractor)包含
    两层卷积,每层卷积后面通过Maxpooling降低分辨率;分类器(classifier)
    由一层全连接层fully-connected layers,FC)+一层输出层构成。
    '''
    #该CNN模型为串行模型
    model = tf.keras.models.Sequential()
    
    #=================== feature extractor ====================
    #第一层卷积需要指定输入图像的shape为[batchsize，32，32，3]
    #输出特征shape：[batchsize，32, 32, 32]
    model.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=(3,3),
                                     input_shape=(32, 32, 3),
                                     activation='relu',
                                     padding='same'))   
    
    
    #将特征图缩小一半，输出shape：[batchsize，16, 16, 32]
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    
    #第二层卷积输出特征shape：[batchsize，16, 16, 64]
    model.add(tf.keras.layers.Conv2D(filters=64,
                                     kernel_size=(3,3),
                                     input_shape=(32, 32, 3),
                                     activation='relu',
                                     padding='same'))
    
    
    #将特征图缩小一半，输出shape：[batchsize，8, 8, 64]
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    
    #将2D的特征图变为向量，输出shape：[batchsize, 4096]
    model.add(tf.keras.layers.Flatten())
    
    #===========--------- classifier ==========================
    #第一层FC， 输出shape:[batchsize, 1024]
    model.add(tf.keras.layers.Dense(1024, activation='relu'))
    
    #随机将第一层FC的30%weights值设置为0，缓解过拟合
    #输出shape：[batchsize，1024]
    model.add(tf.keras.layers.Dropout(rate=0.3))
    
    #第二层FC层作为输出层，10个类别的softmax概率
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    
    return model

In [87]:
#======================导入cifar10数据集 =====================
cifar10 = tf.keras.datasets.cifar10
#C:\Users\XXXX\.keras\datasets下寻找数据集，如没有，则会自动下载
(x_train, y_train),(x_test, y_test) = cifar10.load_data()

In [88]:
#通过打印shape查看数据集基本信息
print('train images:', x_train.shape) #[50000, 32, 32, 3]
print('train labels:', y_train.shape) #[50000, 1]
print('test images:', x_test.shape) #[10000, 32, 32, 3]
print('test labels:', y_test.shape) #[10000, 1]

train images: (50000, 32, 32, 3)
train labels: (50000, 1)
test images: (10000, 32, 32, 3)
test labels: (10000, 1)


In [89]:
#===================== 实例化CNN模型 ========================
model = create_cnn()

print('模型结构为：')
model.summary()


模型结构为：
Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 4096)              0         
_________________________________________________________________
dense_20 (Dense)             (None, 1024)              4195328   
_________________________________________________________________
dropout_10 (Dropout)         (None, 1024)     

In [90]:
#===================== 定义训练超参数 =======================   
model.compile(optimizer='adam', #使用adam优化器
                loss='sparse_categorical_crossentropy', #交叉熵损失
                metrics=['accuracy']) #准确率作为评估性能指标

In [91]:
#=============== 开始迭代训练并保存模型参数=================
#首先创建模型保存的目录    
save_root = 'modelSaved'
if not osp.exists(save_root): os.mkdir(save_root)

#保存模型的文件名，占位符会被自动替换为epoch值和指标值
checkpoint_file = osp.join(save_root,'Cifar10.{epoch:02d}-{val_loss:4f}.h5')
#通过调用该回调函数让模型保存参数
call_backs = [
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_file,
                                        save_weights_only=True, #只保存参数，不保存网络结构 如果要保存网络结构文件会很大
                                        save_freq='epoch')#每次epoch结束才保存模型)
    ]
train_history = model.fit(x_train, y_train, 
                            validation_split=0.2,
                            epochs=100,#训练5个轮次
                            batch_size=80,#每次喂入模型100张图像
                            callbacks=call_backs,
                            verbose=2)#控制打印信息的level

Train on 40000 samples, validate on 10000 samples
Epoch 1/100
40000/40000 - 4s - loss: 3.8481 - accuracy: 0.3156 - val_loss: 1.6148 - val_accuracy: 0.4146
Epoch 2/100
40000/40000 - 3s - loss: 1.5315 - accuracy: 0.4518 - val_loss: 1.4161 - val_accuracy: 0.4971
Epoch 3/100
40000/40000 - 3s - loss: 1.3279 - accuracy: 0.5299 - val_loss: 1.3569 - val_accuracy: 0.5189
Epoch 4/100
40000/40000 - 3s - loss: 1.1656 - accuracy: 0.5892 - val_loss: 1.2781 - val_accuracy: 0.5565
Epoch 5/100
40000/40000 - 3s - loss: 1.0225 - accuracy: 0.6401 - val_loss: 1.3018 - val_accuracy: 0.5556
Epoch 6/100
40000/40000 - 3s - loss: 0.8901 - accuracy: 0.6906 - val_loss: 1.3733 - val_accuracy: 0.5603
Epoch 7/100
40000/40000 - 3s - loss: 0.7638 - accuracy: 0.7350 - val_loss: 1.3307 - val_accuracy: 0.5847
Epoch 8/100
40000/40000 - 3s - loss: 0.6550 - accuracy: 0.7707 - val_loss: 1.4444 - val_accuracy: 0.5856
Epoch 9/100
40000/40000 - 3s - loss: 0.5725 - accuracy: 0.8045 - val_loss: 1.5389 - val_accuracy: 0.5835
Epoch